In [1]:
import yfinance as yf

In [2]:
symbol = "spy"
period = "1d"
interval = "5m"
file_name = f"{symbol}.csv"

In [3]:
ticker = yf.Ticker(symbol)

In [4]:
data = ticker.history(period=period, interval=interval, auto_adjust = True, prepost=True)

In [5]:
data=data.drop(columns=["Dividends", "Stock Splits"])
data['Adj Close']=data['Close']
data.tail()

,Open,High,Low,Close,Volume,Adj Close
Datetime,,,,,,
2021-08-19 11:40:00-04:00,439.315002,440.079987,439.249908,440.078308,561339,440.078308
2021-08-19 11:45:00-04:00,440.059998,440.755005,440.040009,440.670013,1252588,440.670013
2021-08-19 11:50:00-04:00,440.709991,440.809998,440.584686,440.589905,718451,440.589905
2021-08-19 11:55:00-04:00,440.599915,440.890015,440.599915,440.829895,244294,440.829895
2021-08-19 11:57:40-04:00,440.890015,440.890015,440.890015,440.890015,0,440.890015


In [6]:
data.to_csv(file_name)
#cleaned up in vscode to change to "Date Time" and to remove "-04:00"

In [ ]:
INITIAL_BUDGET = 5000
BUDGET_USE = 0.50
RISK_PERCENT = 0

In [7]:
import datetime
import math
market_opentime = datetime.time(9,30,0) #6:30 AM PT
arb_tradestoptime = datetime.time(14,0,0) #11 AM PT

In [8]:
market_opentime

datetime.time(9, 30)

In [9]:

from pyalgotrade import strategy
from pyalgotrade.barfeed import csvfeed
from pyalgotrade.technical import macd

In [10]:
# # Load the bar feed from the CSV file
feed = csvfeed.GenericBarFeed(frequency="minute")
feed.addBarsFromCSV(symbol, file_name)

In [11]:
def safe_round(value, digits):
    if value is not None:
        value = round(value, digits)
    return value

In [12]:
class MyStrategy(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument):
        super(MyStrategy, self).__init__(feed)
        self.__position = None
        #Looking for a MACD crossover
        #self.__sma = ma.SMA(feed[instrument].getCloseDataSeries(), 15)
        self.__macd = macd.MACD(feed[instrument].getCloseDataSeries(), 12, 26, 9)
        self.__instrument = instrument

    def onEnterOk(self, position):
        execInfo = position.getEntryOrder().getExecutionInfo()
        self.info(f"BUY at ${execInfo.getPrice()}")
    
    def onExitOk(self, position):
        execInfo = position.getExitOrder().getExecutionInfo()
        self.info(f"SELL at ${execInfo.getPrice()}")
        self.__position = None
    
    def onBars(self, bars):
        # Wait for enough bars to be available to calculate a SMA.
        if self.__macd.getHistogram()[-1] is None:
            return
        if self.__macd.getHistogram()[-2] is None:
            return
        
        bar = bars[self.__instrument]
        if self.__position is None:
            if market_opentime <= bar.getDateTime().time() <= arb_tradestoptime:
                if self.__macd.getHistogram()[-1]>0 and self.__macd.getHistogram()[-2]<0:
                    print(f"Crossover UP detected at {bar.getDateTime()}. {symbol.upper()} price was ${safe_round(bar.getClose(), 2)}. Buy some {math.ceil(bar.getClose())} CALLS.")
                    self.__position = self.enterLong(self.__instrument, 10, True)                   
                    
                if self.__macd.getHistogram()[-1]<0 and self.__macd.getHistogram()[-2]>0:
                    self.__position = self.enterShort(self.__instrument, 10, True)                    
                    print(f"Crossover DOWN detected at {bar.getDateTime()}. {symbol.upper()} price was ${safe_round(bar.getClose(), 2)}. Buy some {math.floor(bar.getClose())} PUTS.") 

        if self.__position:
            if self.__position.getEntryOrder().getAction() == 4:
                if self.__macd.getHistogram()[-1] > self.__macd.getHistogram()[-2]:
                    print(f"Momentum slowing at {bar.getDateTime()}.")
                    self.__position.exitMarket()
            else:
                if self.__macd.getHistogram()[-1] < self.__macd.getHistogram()[-2]:
                    print(f"Momentum slowing at {bar.getDateTime()}.")
                    self.__position.exitMarket()
                    
        
            
                
            

        #self.info("%s %s" % (bar.getClose(), safe_round(self.__macd.getHistogram()[-1], 4)))

In [13]:
ms1 = MyStrategy(feed, symbol)
ms1.run()

2021-08-19 10:40:00 strategy [INFO] BUY at $437.89898681640625
2021-08-19 10:55:00 strategy [INFO] SELL at $438.3800048828125
2021-08-19 11:25:00 strategy [INFO] BUY at $439.6300048828125
2021-08-19 11:30:00 strategy [INFO] SELL at $438.94000244140625
2021-08-19 11:50:00 strategy [INFO] BUY at $440.7099914550781


Crossover DOWN detected at 2021-08-19 10:35:00. SPY price was $437.9. Buy some 437 PUTS.
Momentum slowing at 2021-08-19 10:50:00.
Crossover UP detected at 2021-08-19 11:20:00. SPY price was $439.65. Buy some 440 CALLS.
Momentum slowing at 2021-08-19 11:25:00.
Crossover UP detected at 2021-08-19 11:45:00. SPY price was $440.67. Buy some 441 CALLS.


In [ ]:
#reverse order:
option time price 
444P @10:01 $0.76    $0.81@10:03
444C @07:26 $0.64    $0.68@07:30
443C @09:21 $0.36    $0.82@12:27
443P @07:01 $0.24    $0.37@07:36
442P @9:40 $0.65     $0.70@09:47
443C @8:35 $0.85     $1.02@08:48
442P @7:35 $0.82     $1.24@07:54
443C @11:00 $
    
445P 6:35 0.40-0.47
446C 6:45 0.28-0.44
445P 7:00 0.30-0.55calc
446C 7:20 0.28-0.36
445P 7:25 0.24-0.34
446C 7:33 0.24
    
    8/17 445C 1.00 OPEn @ 7:25 1.20 High @ 7:29 1.13 Open @ 7:30
    8/17 443P 0.89 Open @ 8:25 1.11 High @ 8:29 1.07 Open @ 8:30
    8/17 443C 1.04 Open @ 10:25 1.25 High @ 10:29 1.17 Open @ 10:35
    
    8/18 444C @6:45 0.54O 1.13H @7:19 1.05O @ 7:25
    8/18 444P @7:50 0.77O 1.16H@8:18 0.99O@8:20   
    8/18 444C @9:15 0.44O  0.46H @9:15 0.42O@9:20
    8/18 444C @9:55 0.28O 0.34H@10:00 0.29O@10:15
    8/18 443P @10:25 0.57O 0.65H@10:27 0.56O@10:30
    8/18 444C @10:35 0.24O  0.62H@11:05 0.38O@11:10
    
    8/19 437P @7:35 1.78 2.22H@7:48 2.14O@7:50
    8/19 440C @8:20 1.85 1.86@8:25 LOSER if SL
    8/19 441C @8:45 1.39O 
                
        
    